# Einführung in Web APIs

Bei einer API (*Application Programming Interface* oder *Programmierschnittstelle*) handelt es sich ganz allgemein erstmal um einen Teil einer Softwarearchitektur, der eine Kommunikation zwischen zwei unterschiedlichen Maschinen ermöglicht. Eine API stellt Entwickler:innen Funktionen bereit, mit denen sie ihre Maschine so programmieren können, dass diese Teile der externen Maschine, die die API bereitstellt, steuern kann.  API-Funktionen haben einen *Namen* und benötigen von Ihnen vergebene *Parameter*. 

Wenn wir im geisteswissenschaftlichen Kontext von APIs sprechen, meinen wir in der Regel *Web APIs*. Hierbei handelt es sich um spezielle APIs, die auf Webservern laufen und meist dazu genutzt werden, strukturierte oder semistrukturierte Datenbestände zu durchsuchen und herunterzuladen -- manchmal sind APIs sogar die einzige Möglichkeit, um an öffentlich bereitgestellte Daten zu kommen. Um eine Web API zu nutzen, benötigen Sie immer zwei, manchmal auch drei Informationen:

* Sie müssen wissen, welche Funktionen die API, mit der Sie arbeiten wollen, bereitstellt. Nur so sind Sie in der Lage, Abfragen zu konstruieren, mit denen Sie genau die Daten aus dem Datenbestand filtern können, die Sie auch tatsächlich benötigen. Eine Auflistung der Funktionen einer API finden Sie in deren Dokumentation.
* Sie benötigen den *Endpoint* der API. Bei einem Endpoint handelt es sich um eine URI (eine Art Webadresse/URL). Über diese Adresse können Sie die Funktionen einer API aufrufen. Der Endpoint der API von *Old Baley Online*, die wir in diesem Notebook nutzen, lautet z.B. [http://www.oldbaileyonline.org/obapi/ob](http://www.oldbaileyonline.org/obapi/ob).
* Manchmal benötigen Sie außerdem noch einen *Access Token*. Hierbei handelt es sich letztlich einfach nur um einen automatisch erzeugten Benutzernamen und ein Passwort. Diese Informationen erhalten Sie vom Anbieter der API. Access Token dienen der Zugriffssteuerung -- manche Anbieter wollen Ihre Daten nicht uneingeschränkt öffentlich zur Verfügung stellen oder zumindest ungefähr nachverfolgen können, von wem bestimmte Zugriffe kommen.

Suchen Sie auf der Webseite des Projekts oder der Institution, an dessen Daten Sie interessiert sind, einfach nach *API* oder *Developers*. Dort finden Sie die Dokumentation sowie alle weiteren Informationen, die Sie für die Verwendung der API benötigen.

## Abfragen konstruieren

In dieser Einheit arbeiten wir mit der Web API von [Old Bailey Online](https://www.oldbaileyonline.org/index.jsp). Das Projekt der Universitäten in Hertfordshire, Sheffield sowie der Open University stellt die erhaltenen Gerichtsprotokolle des Zentralen Strafgerichtshofes von London der Jahre 1674 bis 1913 vollständig digitalisiert und mit Metadaten angereichert zur Verfügung. 

Mit der API des Projekts lassen sich nun einzelne Gerichtsprotokolle nach vordefinierten Kriterien (z.B. Vergehen, Geschlecht von Täter und Opfer, oder Strafmaß) filtern und auswählen. Auf diese Weise können wir mit relativ geringem Programmieraufwand einzelne Texte aus der Datenbank herunterladen und zur Weiterverarbeitung aufbereiten.

**Aufgabe:** Lesen Sie sich die [Dokumentation der API](https://www.oldbaileyonline.org/static/DocAPI.jsp) von Old Bailey Online durch. Werfen Sie dabei auch schonmal einen Blick in die [Übersicht der Funktionen der API](https://www.oldbaileyonline.org/obapi/terms).

**http://www.oldbaileyonline.org/obapi/ob?term0=offcat_violentTheft&term1=vercat_guilty&breakdown=punsubcat&count=10**

Sie können diesen Link einfach in Ihrem Internetbrowser aufrufen um das Ergebnis zu erhalten. Ihnen wird ein JSON angezeigt (das Sie sich auch herunterladen können), wo Sie unter *breakdown* die von uns angefragten Statistiken zu den Strafen und unter *hits* die IDs der von uns benötigten Gerichtsprotokolle finden.

Um besser zu verstehen, wie unsere Abfrage aufgebaut ist, können wir sie in ihre Bestandteile zerlegen:
(Den folgenden Codeblock müssen und können Sie nicht ausführen. Er dient nur der Illustration der Zusammensetzung einer API-Abfrage.)

In [ ]:
http://www.oldbaileyonline.org/obapi/ob 
# Dies ist der Endpoint der API. Hier liegen die 
# verfügbaren Funktionen. Um diese nutzen zu können, müssen Sie Ihre
# Abfrage einfach an die Adresse des Endpoints anhängen.

? 
# Auf den Endpoint folgt ein Fragezeichen. Danach kommt die Abfrage.

term0=offcat_violentTheft 
# Hiermit suchen wir nach allen Fällen, in denen der Vorwurf
# "violent theft" lautet. Syntaktisch beginnt die Abfrage mit dem Namen der Funktion; hinter
# dem Gleichheitszeichen folgt der Parameter. Die genaue Syntax entnehmen Sie der Dokumentation
# einer API.

& 
# Einzelne Funktionsaufrufe werden mit einem kaufmännischen Und getrennt.

term1=vercat_guilty 
# Hiermit filtern wir die Fälle heraus, in denen der oder die
# Angeklagte für schuldig befunden wurde.

&

breakdown=punsubcat 
# Hiermit fragen wir ab welche und wie oft bestimmte Strafen verhängt wurden.

&

count=10  
# Die maximale Anzahl der Ergebnisse (also der Protokolle) die uns angezeigt
# werden soll. Fast alle Anbieter legen hier Beschränkungen auf; bei Old Bailey Online lassen
# sich nicht mehr als 1.000 Protokolle in einer Abfrage zurückgeben. Auf diese Weise soll eine Überlastung
# der Server durch zu umfangreiche Rückgaben vermieden werden. Wie Sie trotzdem alle zu
# Ihrer Abfrage passenden Daten herunterladen können, sehen wir uns weiter hinten im Notebook an.

## APIs mit Python abfragen

Idealerweise möchten wir die Daten natürlich automatisiert herunterladen und direkt weiterverarbeiten. In Python lassen sich API-Abfragen z.B. mit der `requests`-Bibliothek durchführen.

Zuerst importieren wir, wie üblich, die benötigten Programmbibliotheken. Die meisten davon kennen Sie ja bereits. Bei `pprint` (pretty-print) handelt es sich um eine Alternative zur Funktion `print()` mit der Sie sich strukturierte Daten, wie zum Beispiel
 JSONs, bzw. Dictionaries formatiert (und damit lesbarer) anzeigen lassen können.

In [ ]:
import json
import pprint

Es folgt nun die Anfrage an die API über den Endpoint:

In [ ]:
# Der Endpoint von Old Baily Online
api_base_url = 'http://www.oldbaileyonline.org/obapi/ob'

# Hier konstruieren wir die Abfrage als String. Die Abfrage
# ist identisch zu der im vorletzten Codeblock.
query_url = api_base_url + '?term0=offcat_violentTheft'
query_url += '&term1=vercat_guilty'
query_url += '&breakdown=punsubcat'
query_url += '&count=10'

# Hier wird die Abfrage an den Server von Old Baily Online gesandt.
response = requests.get(query_url)

if response.status_code == 200:
  response_content = response.json()
else:
  response_content = None

pprint.pprint(response_content)

Mit der Funktion `json()` der `requests`-Bibliothek können Sie das JSON als Dictionary parsen. Um Programmfehler zu vermeiden haben wir in diesem Beispiel außerdem eine if-Abfrage zur Sicherung eingebaut. Der Variable `response_content` wird nur dann ein Wert zugewiesen, wenn unsere Anfrage an den Server auch tatsächlich mit Daten beantwortet wird. Bei jeder Anfrage an einen Server gibt dieser einen *HTTP-Statuscode* zurück. Auf diese Weise lassen sich Fehler eingrenzen -- so legt ein Statuscode z.B. fest, ob der Fehler auf Client- oder Server-Seite liegt. Einen Überblick über alle HTTP-Statuscodes finden Sie z.B. [hier](https://de.wikipedia.org/wiki/HTTP-Statuscode) oder [hier](https://http.cat/).

## Abfrage mit Header

Eine Alternative ist die API-Abfrage mit einem *Header*. Dies bietet sich insbesondere an, wenn Abfragen besonders komplex sind oder deren Bestandteile von Ihrem Python-Programm gesteuert werden sollen. Für das `requests`-Modul wird ein Header als Dictionary definiert und der Funktion `get()` zusammen mit dem Endpoint der API als Parameter übergeben. Durch die Verwendung eines Dictionaries sparen Sie sich die Hantierung mit komplexen Strings, wie im oberen Beispiel. Ansonsten unterscheidet sich der folgende Codeblock nur marginal vom vorigen.

In [ ]:
api_base_url = 'http://www.oldbaileyonline.org/obapi/ob'

request_header = {
    'term0': 'offcat_violentTheft',
    'term1': 'vercat_guilty',
    'breakdown': 'punsubcat',
    'count': '10',
    'start': '0'
}

response = requests.get(api_base_url, request_header)

if response.status_code == 200:
  response_content = response.json()
else:
  response_content = None

pprint.pprint(response_content)

Zu unserer API-Abfrage ist nur der Parameter `start` hinzugekommen. Wie bereits weiter oben beschrieben beschränken die meisten APIs die Anzahl der Ergebnisse, die sich bei einer Abfrage herunterladen lassen. Um trotzdem alle möglichen Ergebnisse erhalten zu können, bieten viele APIs eine virtuelle Paginierung an (Details finden Sie in der jeweiligen Dokumentation). Im obigen Beispiel legen wir fest, dass wir nur 10 Ergebnisse der insgesamt 5760 Treffer unserer Abfrage zurückbekommen wollen. Mit `start=0` legen wir fest, dass das erste dieser zehn Teilergebnisse auch das erste Ergebnis aller möglichen Treffer sein soll. Würden wir `start=4000` setzen, würden wir die Ergebnisse 4001 bis 4011 aller möglichen Treffer zurückbekommen. Möchten wir die Protokoll-IDs aller 5760 Treffer erhalten, müssen wir die API-Abfrage mehrmals an den Server senden und dabei `start` jedes Mal entsprechend erhöhen.

Um die Auslastung der Server des API-Anbieters möglichst gering zu halten, empfiehlt es sich nach jeder Abfrage eine kurze Wartepause einzulegen, bevor man mit der nächsten fortfährt. In Python kann man solche Pausen mit Hilfe der Funktion `sleep()` aus der Bibliothek `time` sehr leicht implementieren:

In [ ]:
import time

print('Warte 2 Sekunden')
time.sleep(2)
print('Fertig')

## Aufgabe
Entwickeln Sie eine eigene Abfrage, mit der Sie sich die IDs der Gerichtsprotokolle von Old Bailey Online nach von Ihnen gewählten Kriterien zurückgeben lassen können. Speichern Sie alle(!) IDs in einer Liste. Berücksichtigen Sie, dass Sie zwischen den Abfragen eine kurze Wartepause einbauen.

In [ ]:
# your code

## Daten herunterladen

Nun haben wir eine Liste von IDs, müssen aber natürlich auch noch die Gerichtsprotokolle an sich herunterladen. Um herauszufinden, wie wir an die Texte selbst kommen, können wir wieder die Dokumentation der API von Old Bailey Online konsultieren. Zum Download einzelner Texte wird ein eigener Endpoint bereitgestellt. So können wir den ersten Text aus unserer ersten Abfrage z.B. auf folgende Weise herunterladen:

In [ ]:
api_base_url = 'http://www.oldbaileyonline.org/obapi/text'

request_header = {
    'div': 't16751013-5'
}

response = requests.get(api_base_url, request_header)

if response.status_code == 200:
  response_content = response.text

print(response_content)

Das Ergebnis erhalten Sie als XML-Datei. Zum Vergleich können Sie sich das Protokoll auf der [Webseite](https://www.oldbaileyonline.org/browse.jsp?id=ft18110529-18&div=t16751013-5) ansehen. Die API gibt Ihnen das Dokument als XML zurück. XML ist, ähnlich wie HTML, eine Auszeichnungssprache mit der bestimmte Elemente eines Dokuments maschinenlesbar gemacht werden können. Tags und deren Attribute funktionieren in XML analog wie bei HTML. Sie können die XML-Struktur eines Dokuments oftmals auch dazu nutzen, um mit geringem Aufwand Informationen auszulesen. Im Fall der Gerichtsprotokolle sind unter anderem Personennamen, Ortsnamen und Straftatbestände als solche in den Texten mit eigenen XML-Tags ausgezeichnet. Sie können solche Informationen mit Hilfe von BeautifulSoup extrahieren.

Wie das Auslesen dieser Informationen bei einer  gewöhnlichen Webseite in HTML mit Hilfe von BeautifulSoup funktioniert, werden wir in den nächsten Abschnitten durchgehen. Zunächst noch ein kurzer Exkurs zu den Grundlagen von HTML.